In [1]:

import mobster as mb





Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import pickle

data = mb.load_example_data("./")

data = data["1:1"]

In [3]:
fitted = mb.fit_mobster(data, 1,1, max_it = 500, lr = 0.01,  e = 0.00001)

Running MOBSTER on 3 karyotypes with 1 subclones.
Fitting a model with tail
Computing cluster assignements.
Computing information criteria.
Done!


ELBO: -5812.814148681  :  86%|████████▌ | 429/500 [00:15<00:02, 27.11it/s]


In [4]:
mb.plot_results(data, fitted, output= "cane.png")

In [5]:
fitted["loss"]


array([-1574.98770983,  -893.67850719,   425.15445144, -1741.34472422,
       -1166.44859113,  -304.400311  , -2235.97032374, -2092.49880096,
       -2404.12979616,  -683.96267986, -1965.34832134,   737.27083333,
       -2204.20788369,  -108.63953837, -1662.55995204,  -866.75419664,
       -1591.35431655, -2029.50269784, -1862.18420264, -1497.53267386,
       -2836.867506  , -2897.92026379, -2884.4367506 , -2000.84442446,
       -3026.70293765, -3048.11870504, -2065.01618705, -3158.21582734,
       -2225.20953237, -2728.867506  , -3320.74730216, -3305.98081535,
       -3070.808753  , -3137.61151079, -3686.89388489, -3789.24340528,
       -3657.94634293, -3620.77128297, -3847.65617506, -3882.10641487,
       -3519.32104317, -3912.05065947, -4131.05665468, -4060.35641487,
       -4041.92775779, -3670.11990408, -4201.34052758, -4315.70953237,
       -4362.21193046, -4466.36121103, -4481.26139089, -4544.7146283 ,
       -4498.2853717 , -4648.37979616, -4686.76199041, -4739.29346523,
      

In [10]:
fitted["model_parameters"]

{'1:0': {'cluster_probs': array([[4.1709416e-02, 3.6734819e-02, 4.7473140e-02, ..., 8.8347407e-04,
          5.1930570e-04, 5.3448504e-04],
         [1.1838845e-10, 3.7147260e-16, 4.9094347e-17, ..., 9.9911648e-01,
          9.9948061e-01, 9.9946558e-01],
         [9.5829052e-01, 9.6326512e-01, 9.5252687e-01, ..., 4.7583073e-35,
          4.0837160e-26, 3.3926188e-25]], dtype=float32),
  'cluster_assignments': array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0,
         0, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 0, 1, 0, 1, 1,
         0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 1, 2, 0, 2, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
         1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1,
         1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 